<a href="https://colab.research.google.com/github/jminangods/analisis-data/blob/main/RAG_Completo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================================================
# PASO 1: WEB SCRAPER ROBUSTO PARA MÚLTIPLES PÁGINAS
# ============================================================================

# Instalaciones para Colab
!pip install -q requests beautifulsoup4 fpdf2 sentence-transformers faiss-cpu torch
!pip install -q lxml html5lib  # Parsers adicionales para BeautifulSoup

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import requests
from bs4 import BeautifulSoup
from fpdf import FPDF
import time
import re
from urllib.parse import urljoin, urlparse
from typing import List, Dict, Optional
import warnings
warnings.filterwarnings("ignore")

In [7]:
class WebScraperToPDF:
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        })
        self.scraped_content = []

    def clean_text(self, text: str) -> str:
        """Limpiar texto extraído"""
        # Remover espacios extra y saltos de línea
        text = re.sub(r'\s+', ' ', text)
        # Remover caracteres especiales problemáticos
        text = re.sub(r'[^\w\s\.\,\;\:\!\?\-\(\)áéíóúñÁÉÍÓÚÑ]', '', text)
        return text.strip()

    def scrape_single_page(self, url: str, max_length: int = 5000) -> Dict:
        """Scrapear una página individual"""
        print(f"🌐 Scrapeando: {url}")

        try:
            response = self.session.get(url, timeout=15)
            response.raise_for_status()
            response.encoding = 'utf-8'

            soup = BeautifulSoup(response.content, 'html.parser')

            # Remover elementos no deseados
            for element in soup(['script', 'style', 'nav', 'footer', 'header', 'aside']):
                element.decompose()

            # Extraer título
            title = soup.find('title')
            title = title.get_text().strip() if title else f"Página de {urlparse(url).netloc}"

            # Extraer contenido principal
            # Buscar contenedores comunes de contenido
            content_selectors = [
                'main', 'article', '.content', '.post', '.entry-content',
                '[role="main"]', '.main-content', '#content', '.page-content'
            ]

            content_element = None
            for selector in content_selectors:
                content_element = soup.select_one(selector)
                if content_element:
                    break

            # Si no encuentra contenedor específico, usar body
            if not content_element:
                content_element = soup.find('body')

            if content_element:
                # Extraer texto de párrafos, headers, listas
                text_elements = content_element.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'li', 'div'])

                content_parts = []
                for element in text_elements:
                    text = element.get_text().strip()
                    if len(text) > 20:  # Solo textos significativos
                        content_parts.append(text)

                content = '\n\n'.join(content_parts)
            else:
                content = soup.get_text()

            # Limpiar y truncar contenido
            content = self.clean_text(content)
            if len(content) > max_length:
                content = content[:max_length] + "..."

            print(f"✅ Extraído: {len(content)} caracteres")

            return {
                'url': url,
                'title': title,
                'content': content,
                'success': True,
                'length': len(content)
            }

        except Exception as e:
            print(f"❌ Error en {url}: {str(e)}")
            return {
                'url': url,
                'title': f"Error: {urlparse(url).netloc}",
                'content': f"No se pudo extraer contenido de {url}. Error: {str(e)}",
                'success': False,
                'length': 0
            }

    def scrape_multiple_pages(self, urls: List[str], delay: float = 1.0) -> List[Dict]:
        """Scrapear múltiples páginas con delay"""
        print(f"🚀 Iniciando scraping de {len(urls)} páginas...")

        results = []
        for i, url in enumerate(urls, 1):
            print(f"\n📄 Página {i}/{len(urls)}")

            result = self.scrape_single_page(url)
            results.append(result)
            self.scraped_content.append(result)

            # Delay entre requests para ser respetuoso
            if i < len(urls):
                print(f"⏳ Esperando {delay}s...")
                time.sleep(delay)

        successful = sum(1 for r in results if r['success'])
        total_chars = sum(r['length'] for r in results)

        print(f"\n✅ Scraping completado!")
        print(f"   📊 Páginas exitosas: {successful}/{len(urls)}")
        print(f"   📝 Total de caracteres: {total_chars:,}")

        return results

    def create_pdf(self, output_filename: str = "scraped_content.pdf") -> str:
        """Crear PDF con todo el contenido scrapeado"""
        print(f"📄 Creando PDF: {output_filename}")

        if not self.scraped_content:
            print("❌ No hay contenido para crear PDF")
            return None

        # Configurar PDF
        pdf = FPDF()
        pdf.set_auto_page_break(auto=True, margin=15)

        # Página de título
        pdf.add_page()
        pdf.set_font('Arial', 'B', 16)
        pdf.cell(0, 10, 'Contenido Web Consolidado', 0, 1, 'C')
        pdf.set_font('Arial', '', 10)
        pdf.cell(0, 10, f'Generado automáticamente - {len(self.scraped_content)} páginas', 0, 1, 'C')
        pdf.ln(10)

        # Índice
        pdf.set_font('Arial', 'B', 14)
        pdf.cell(0, 10, 'Índice', 0, 1, 'L')
        pdf.set_font('Arial', '', 10)

        for i, page_data in enumerate(self.scraped_content, 1):
            title = page_data['title'][:60] + "..." if len(page_data['title']) > 60 else page_data['title']
            try:
                pdf.cell(0, 6, f"{i}. {title}", 0, 1, 'L')
            except:
                pdf.cell(0, 6, f"{i}. [Título con caracteres especiales]", 0, 1, 'L')

        # Contenido de cada página
        for i, page_data in enumerate(self.scraped_content, 1):
            pdf.add_page()

            # Título de la sección
            pdf.set_font('Arial', 'B', 14)
            title = page_data['title']
            try:
                pdf.cell(0, 10, f"{i}. {title}", 0, 1, 'L')
            except:
                pdf.cell(0, 10, f"{i}. [Título con caracteres especiales]", 0, 1, 'L')

            # URL
            pdf.set_font('Arial', 'I', 8)
            pdf.cell(0, 5, f"Fuente: {page_data['url']}", 0, 1, 'L')
            pdf.ln(5)

            # Contenido
            pdf.set_font('Arial', '', 10)
            content = page_data['content']

            # Dividir contenido en líneas manejables
            try:
                # Intentar escribir contenido normal
                lines = content.split('\n')
                for line in lines:
                    if line.strip():
                        # Manejar líneas largas
                        words = line.split(' ')
                        current_line = ""
                        for word in words:
                            if len(current_line + word) < 80:
                                current_line += word + " "
                            else:
                                if current_line:
                                    try:
                                        pdf.cell(0, 5, current_line.strip(), 0, 1, 'L')
                                    except:
                                        pdf.cell(0, 5, "[Línea con caracteres especiales]", 0, 1, 'L')
                                current_line = word + " "
                        if current_line:
                            try:
                                pdf.cell(0, 5, current_line.strip(), 0, 1, 'L')
                            except:
                                pdf.cell(0, 5, "[Línea con caracteres especiales]", 0, 1, 'L')
                    pdf.ln(2)
            except Exception as e:
                pdf.cell(0, 5, f"[Error mostrando contenido: {str(e)}]", 0, 1, 'L')

            pdf.ln(10)

        # Guardar PDF
        try:
            pdf.output(output_filename)
            print(f"✅ PDF creado exitosamente: {output_filename}")
            return output_filename
        except Exception as e:
            print(f"❌ Error creando PDF: {e}")
            return None


In [8]:
# ============================================================================
# FUNCIÓN HELPER PARA URLS FÁCILES
# ============================================================================

def quick_scrape_to_pdf(urls: List[str], filename: str = "content.pdf", delay: float = 1.0) -> str:
    """Función rápida para scrapear URLs y crear PDF"""
    scraper = WebScraperToPDF()
    scraper.scrape_multiple_pages(urls, delay=delay)
    return scraper.create_pdf(filename)

In [17]:
def demo_web_scraper():
    """Demo del web scraper"""
    print("🧪 DEMO WEB SCRAPER")
    print("="*50)

    # URLs de ejemplo (reemplaza con las tuyas)
    #test_urls = [
    #    "https://ister.edu.ec/",
    #    "https://ister.edu.ec/nosotros/",
    #    "https://ister.edu.ec/informe-rendicion-de-cuentas/",
    #    "https://ister.edu.ec/informe-rendicion-de-cuentas/#",
    #    "https://ister.edu.ec/codigo-de-etica/",
    #    "https://ister.edu.ec/normativa-general/",
    #    "https://ister.edu.ec/normativa-institucional/",
    #    "https://ister.edu.ec/programas-de-posgrado/",
    #    "https://ister.edu.ec/oferta-tecnologias-superiores/",
    #    "https://ister.edu.ec/oferta-tecnologias-superior/",
    #    "https://ister.edu.ec/oferta-tecnicaturas-superiores/",
    #    "https://ister.edu.ec/campus-norte/",
    #    "https://ister.edu.ec/campus-sur/",
    #    "https://ister.edu.ec/brochure-informativo/",
    #    "https://ister.edu.ec/centros-de-apoyo/",
    #    "https://ister.edu.ec/requisitos/",
    #    "https://ister.edu.ec/financiamiento/",
    #    "https://ister.edu.ec/homologacion-y-reingreso/",
    #    "https://ister.edu.ec/vinculacion-universitario-ruminahui/",
    #    "https://ister.edu.ec/programas-y-proyectos/",
    #    "https://ister.edu.ec/practicas-pre-profesionales/",
    #    "https://ister.edu.ec/educacion-continua-universitario-ruminahui/",
    #    "https://ister.edu.ec/comunidad/",
    #    "https://ister.edu.ec/bolsa-de-empleo/",
    #    "https://ister.edu.ec/investigacion/",
    #    "https://ister.edu.ec/investigacion/normativa-de-investigacion/",
    #    "https://ister.edu.ec/investigacion/lineas-de-investigacion/",
    #    "https://ister.edu.ec/proyectos-de-investigacion/",
    #    "https://ister.edu.ec/investigacion/proyectos-de-investigacion/",
    #    "https://ister.edu.ec/plan-operativo-anual/",
    #    "https://ister.edu.ec/planificacion-institucional/",
    #    "https://ister.edu.ec/bienestar-institucional-2/",
    #    "https://ister.edu.ec/becas-y-ayudas-economicas/",
    #    "https://ister.edu.ec/plan-de-mejoras-2024/",
    #    "https://ister.edu.ec/plan-de-autoevaluacion/",
    #    "https://ister.edu.ec/reglamento-de-aseguramiento-interno-de-la-calidad-del-universitario-ruminahui/",
    #    "https://ister.edu.ec/institucion-acreditada/",
    #    "https://ister.edu.ec/resultado-de-la-autoevaluacion/",
    #    "https://ister.edu.ec/universidades-para-intercambios-internacionales-para-estudiantes-y-docentes/"
    #]


    test_urls = [
         "https://es.wikipedia.org/wiki/Inteligencia_artificial",
        "https://es.wikipedia.org/wiki/Aprendizaje_automático",
        "https://es.wikipedia.org/wiki/Red_neuronal_artificial"
    ]

    print("🌐 URLs de prueba:")
    for i, url in enumerate(test_urls, 1):
        print(f"  {i}. {url}")

    # Crear scraper
    scraper = WebScraperToPDF()

    # Scrapear páginas
    results = scraper.scrape_multiple_pages(test_urls, delay=1.0)

    # Crear PDF
    pdf_file = scraper.create_pdf("demo_content.pdf")

    return scraper, pdf_file

print("🚀 Web Scraper listo!")
print("\n💡 Para usar:")
print("scraper = WebScraperToPDF()")
print("urls = ['url1', 'url2', 'url3']")
print("scraper.scrape_multiple_pages(urls)")
print("pdf_file = scraper.create_pdf('mi_contenido.pdf')")
print("\n🚀 O usa la función rápida:")
print("pdf_file = quick_scrape_to_pdf(['url1', 'url2'], 'content.pdf')")

# Ejecutar demo automáticamente
print("🚀 Ejecutando demo...")
demo_scraper, demo_pdf = demo_web_scraper()

# Verificar si el PDF se creó
import os
if demo_pdf and os.path.exists(demo_pdf):
    print(f"\n🎉 ¡PDF creado exitosamente!")
    print(f"📁 Ubicación: {demo_pdf}")
    print(f"📊 Tamaño: {os.path.getsize(demo_pdf)} bytes")

    # Mostrar contenido del directorio actual
    print(f"\n📂 Archivos en el directorio actual:")
    files = [f for f in os.listdir('.') if f.endswith('.pdf')]
    for file in files:
        print(f"   📄 {file}")
else:
    print("❌ No se pudo crear el PDF")

print("\n💡 Para descargar el PDF en Colab:")
print("from google.colab import files")
print("files.download('demo_content.pdf')")

🚀 Web Scraper listo!

💡 Para usar:
scraper = WebScraperToPDF()
urls = ['url1', 'url2', 'url3']
scraper.scrape_multiple_pages(urls)
pdf_file = scraper.create_pdf('mi_contenido.pdf')

🚀 O usa la función rápida:
pdf_file = quick_scrape_to_pdf(['url1', 'url2'], 'content.pdf')
🚀 Ejecutando demo...
🧪 DEMO WEB SCRAPER
🌐 URLs de prueba:
  1. https://es.wikipedia.org/wiki/Inteligencia_artificial
  2. https://es.wikipedia.org/wiki/Aprendizaje_automático
  3. https://es.wikipedia.org/wiki/Red_neuronal_artificial
🚀 Iniciando scraping de 3 páginas...

📄 Página 1/3
🌐 Scrapeando: https://es.wikipedia.org/wiki/Inteligencia_artificial
✅ Extraído: 5003 caracteres
⏳ Esperando 1.0s...

📄 Página 2/3
🌐 Scrapeando: https://es.wikipedia.org/wiki/Aprendizaje_automático
✅ Extraído: 5003 caracteres
⏳ Esperando 1.0s...

📄 Página 3/3
🌐 Scrapeando: https://es.wikipedia.org/wiki/Red_neuronal_artificial
✅ Extraído: 5003 caracteres

✅ Scraping completado!
   📊 Páginas exitosas: 3/3
   📝 Total de caracteres: 15,009
📄 C

In [12]:
# ============================================================================
# PASO 2: SISTEMA RAG CON PDF GENERADO
# ============================================================================

# Instalaciones adicionales para PDF
!pip install -q PyPDF2 pdfplumber
!pip install -q chromadb  # Vector database fácil para PoC

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 6.8 MB/s eta 0:00

In [13]:
import PyPDF2
import pdfplumber
import numpy as np
import faiss
import torch
from sentence_transformers import SentenceTransformer
import chromadb
from typing import List, Dict
import re
import os
import warnings
warnings.filterwarnings("ignore")

In [18]:
class PDFRAGSystem:
    def __init__(self, use_vector_db=True):
        print("🚀 Inicializando Sistema RAG para PDF...")

        # Modelo de embeddings
        self.embedder = SentenceTransformer('all-MiniLM-L6-v2')
        print("✅ Modelo de embeddings cargado")

        # Configurar almacenamiento
        self.use_vector_db = use_vector_db
        self.chunks = []

        if use_vector_db:
            print("📊 Configurando ChromaDB...")
            self.client = chromadb.Client()
            try:
                self.client.delete_collection("pdf_docs")
            except:
                pass  # Collection no existe
            self.collection = self.client.create_collection(
                name="pdf_docs",
                metadata={"hnsw:space": "cosine"}
            )
        else:
            print("📁 Configurando FAISS...")
            self.index = None
            self.metadata = []

        print("✅ Sistema RAG listo!")

    def extract_text_from_pdf(self, pdf_path: str) -> List[Dict]:
        """Extraer texto del PDF de manera inteligente"""
        print(f"📄 Extrayendo texto de: {pdf_path}")

        if not os.path.exists(pdf_path):
            print(f"❌ Archivo no encontrado: {pdf_path}")
            return []

        chunks = []

        try:
            # Método 1: Usar pdfplumber (mejor para texto estructurado)
            with pdfplumber.open(pdf_path) as pdf:
                for page_num, page in enumerate(pdf.pages):
                    text = page.extract_text()

                    if text and len(text.strip()) > 50:
                        # Limpiar texto
                        text = self._clean_extracted_text(text)

                        # Dividir en chunks inteligentes
                        page_chunks = self._smart_text_splitting(text, page_num + 1)
                        chunks.extend(page_chunks)

            print(f"✅ Texto extraído: {len(chunks)} chunks de {len(pdf.pages)} páginas")

        except Exception as e:
            print(f"⚠️ Error con pdfplumber, intentando PyPDF2: {e}")

            # Método 2: Backup con PyPDF2
            try:
                with open(pdf_path, 'rb') as file:
                    pdf_reader = PyPDF2.PdfReader(file)

                    for page_num, page in enumerate(pdf_reader.pages):
                        text = page.extract_text()

                        if text and len(text.strip()) > 50:
                            text = self._clean_extracted_text(text)
                            page_chunks = self._smart_text_splitting(text, page_num + 1)
                            chunks.extend(page_chunks)

                print(f"✅ Texto extraído con PyPDF2: {len(chunks)} chunks")

            except Exception as e2:
                print(f"❌ Error extrayendo PDF: {e2}")
                return []

        return chunks

    def _clean_extracted_text(self, text: str) -> str:
        """Limpiar texto extraído del PDF"""
        # Remover saltos de línea excesivos
        text = re.sub(r'\n+', '\n', text)
        # Remover espacios múltiples
        text = re.sub(r' +', ' ', text)
        # Remover caracteres extraños comunes en PDFs
        text = re.sub(r'[^\w\s\.\,\;\:\!\?\-\(\)áéíóúñÁÉÍÓÚÑ\n]', ' ', text)
        return text.strip()

    def _smart_text_splitting(self, text: str, page_num: int, chunk_size: int = 500) -> List[Dict]:
        """Dividir texto de manera inteligente"""
        chunks = []

        # Dividir por párrafos primero
        paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]

        current_chunk = ""
        chunk_counter = 1

        for paragraph in paragraphs:
            # Si el párrafo cabe en el chunk actual
            if len(current_chunk) + len(paragraph) < chunk_size:
                current_chunk += paragraph + "\n\n"
            else:
                # Guardar chunk actual si no está vacío
                if current_chunk.strip():
                    chunks.append({
                        'content': current_chunk.strip(),
                        'page': page_num,
                        'chunk': chunk_counter,
                        'source': 'pdf',
                        'type': 'text'
                    })
                    chunk_counter += 1

                # Iniciar nuevo chunk
                current_chunk = paragraph + "\n\n"

        # Guardar último chunk
        if current_chunk.strip():
            chunks.append({
                'content': current_chunk.strip(),
                'page': page_num,
                'chunk': chunk_counter,
                'source': 'pdf',
                'type': 'text'
            })

        return chunks

    def index_pdf(self, pdf_path: str):
        """Indexar contenido del PDF"""
        print(f"🔍 Indexando PDF: {pdf_path}")

        # Extraer chunks del PDF
        chunks = self.extract_text_from_pdf(pdf_path)

        if not chunks:
            print("❌ No se pudo extraer contenido del PDF")
            return False

        self.chunks = chunks

        # Generar embeddings
        print("🧠 Generando embeddings...")
        texts = [chunk['content'] for chunk in chunks]
        embeddings = self.embedder.encode(texts, show_progress_bar=True)

        if self.use_vector_db:
            # ChromaDB
            ids = [f"chunk_{i}" for i in range(len(chunks))]
            metadatas = [{k: v for k, v in chunk.items() if k != 'content'} for chunk in chunks]

            self.collection.add(
                embeddings=embeddings.tolist(),
                documents=texts,
                metadatas=metadatas,
                ids=ids
            )
            print(f"✅ {len(chunks)} chunks indexados en ChromaDB")
        else:
            # FAISS
            dimension = embeddings.shape[1]
            self.index = faiss.IndexFlatL2(dimension)
            self.index.add(embeddings.astype('float32'))
            self.metadata = chunks
            print(f"✅ {len(chunks)} chunks indexados en FAISS")

        return True

    def search(self, query: str, k: int = 3) -> List[Dict]:
        """Buscar chunks relevantes"""
        if not self.chunks:
            print("⚠️ No hay contenido indexado")
            return []

        # Generar embedding de la consulta
        query_embedding = self.embedder.encode([query])

        if self.use_vector_db:
            # ChromaDB
            results = self.collection.query(
                query_embeddings=query_embedding.tolist(),
                n_results=min(k, len(self.chunks))
            )

            search_results = []
            for i in range(len(results['documents'][0])):
                search_results.append({
                    'content': results['documents'][0][i],
                    'metadata': results['metadatas'][0][i],
                    'score': results['distances'][0][i]
                })
        else:
            # FAISS
            scores, indices = self.index.search(query_embedding.astype('float32'), min(k, len(self.chunks)))

            search_results = []
            for i in range(len(indices[0])):
                if indices[0][i] < len(self.chunks):
                    search_results.append({
                        'content': self.chunks[indices[0][i]]['content'],
                        'metadata': self.chunks[indices[0][i]],
                        'score': float(scores[0][i])
                    })

        return search_results

    def generate_answer(self, query: str, search_results: List[Dict]) -> str:
        """Generar respuesta basada en el contexto"""
        if not search_results:
            return "No se encontró información relevante en el PDF para responder la pregunta."

        # Construir contexto
        context_parts = []
        sources = []

        for result in search_results[:3]:  # Top 3 resultados
            content = result['content']
            metadata = result['metadata']

            context_parts.append(content)
            sources.append(f"(Página {metadata['page']}, Chunk {metadata['chunk']})")

        context = " ".join(context_parts)

        # Análisis simple de la consulta para generar respuesta
        query_lower = query.lower()
        context_lower = context.lower()

        # Buscar respuestas específicas en el contexto
        sentences = [s.strip() + '.' for s in context.split('.') if s.strip()]
        relevant_sentences = []

        # Palabras clave de la consulta
        query_words = [word for word in query_lower.split() if len(word) > 3]

        for sentence in sentences:
            sentence_lower = sentence.lower()
            # Contar coincidencias de palabras clave
            matches = sum(1 for word in query_words if word in sentence_lower)
            if matches > 0:
                relevant_sentences.append((sentence, matches))

        # Ordenar por relevancia
        relevant_sentences.sort(key=lambda x: x[1], reverse=True)

        if relevant_sentences:
            # Tomar las 2-3 oraciones más relevantes
            answer_parts = [sent[0] for sent in relevant_sentences[:3]]
            answer = " ".join(answer_parts)
        else:
            # Respuesta por defecto con las primeras oraciones del contexto
            answer = ". ".join(sentences[:2]) + "."

        # Agregar fuentes
        answer += f" Fuentes: {', '.join(sources[:2])}"

        return answer

    def query(self, question: str, k: int = 3, verbose: bool = True) -> Dict:
        """Consulta RAG completa"""
        if verbose:
            print(f"\n❓ Pregunta: {question}")

        # Buscar contenido relevante
        search_results = self.search(question, k)

        if verbose and search_results:
            print(f"📚 Encontrados {len(search_results)} chunks relevantes:")
            for i, result in enumerate(search_results[:2]):
                meta = result['metadata']
                print(f"  {i+1}. Página {meta['page']}, Chunk {meta['chunk']} (Score: {result['score']:.3f})")

        # Generar respuesta
        answer = self.generate_answer(question, search_results)

        if verbose:
            print(f"💡 Respuesta: {answer}")

        return {
            'question': question,
            'answer': answer,
            'sources': search_results,
            'context': " ".join([r['content'] for r in search_results[:2]])
        }

# ============================================================================
# FUNCIÓN RÁPIDA PARA USAR CON EL PDF GENERADO
# ============================================================================

def quick_pdf_rag(pdf_path: str = "demo_content.pdf"):
    """Función rápida para crear RAG con el PDF"""
    print("🚀 Configurando RAG rápido para PDF...")

    # Verificar que el PDF existe
    if not os.path.exists(pdf_path):
        print(f"❌ PDF no encontrado: {pdf_path}")
        print("📂 PDFs disponibles:")
        pdfs = [f for f in os.listdir('.') if f.endswith('.pdf')]
        for pdf in pdfs:
            print(f"   📄 {pdf}")
        return None

    # Crear sistema RAG
    rag = PDFRAGSystem(use_vector_db=True)

    # Indexar PDF
    success = rag.index_pdf(pdf_path)

    if not success:
        print("❌ Error indexando PDF")
        return None

    print("✅ RAG listo para consultas!")
    return rag

In [19]:
# ============================================================================
# DEMO AUTOMÁTICO
# ============================================================================

print("🧪 INICIANDO DEMO RAG CON PDF")
print("="*50)

# Crear RAG con el PDF generado
pdf_rag = quick_pdf_rag("demo_content.pdf")

if pdf_rag:
    # Preguntas de prueba
    test_questions = [
        "¿Qué es la inteligencia artificial?",
        "¿Cómo funciona el aprendizaje automático?",
        "¿Qué son las redes neuronales?",
        "¿Cuáles son las aplicaciones de la IA?"
    ]

    print(f"\n🧪 PROBANDO {len(test_questions)} PREGUNTAS")
    print("="*50)

    for i, question in enumerate(test_questions, 1):
        print(f"\n--- Pregunta {i} ---")
        result = pdf_rag.query(question)
        print("-" * 40)

    print("\n🎉 Demo completado!")
    print("\n💡 Para usar:")
    print("result = pdf_rag.query('tu pregunta aquí')")
    print("print(result['answer'])")
else:
    print("❌ No se pudo configurar el RAG. Verifica que el PDF existe.")

🧪 INICIANDO DEMO RAG CON PDF
🚀 Configurando RAG rápido para PDF...
🚀 Inicializando Sistema RAG para PDF...
✅ Modelo de embeddings cargado
📊 Configurando ChromaDB...
✅ Sistema RAG listo!
🔍 Indexando PDF: demo_content.pdf
📄 Extrayendo texto de: demo_content.pdf
✅ Texto extraído: 7 chunks de 7 páginas
🧠 Generando embeddings...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ 7 chunks indexados en ChromaDB
✅ RAG listo para consultas!

🧪 PROBANDO 4 PREGUNTAS

--- Pregunta 1 ---

❓ Pregunta: ¿Qué es la inteligencia artificial?
📚 Encontrados 3 chunks relevantes:
  1. Página 2, Chunk 1 (Score: 0.346)
  2. Página 1, Chunk 1 (Score: 0.437)
💡 Respuesta: Inteligencia artificial - Wikipedia, la enciclopedia libre
Fuente: https:  es. org wiki Inteligencia_artificial
De Wikipedia, la enciclopedia libre Imagen generada por la inteligencia
artificial Dalle 3. Vídeo explicativo de 6:47 min, en idioma euskera (con
subtítulos en castellano) sobre la inteligencia artificial, incluyendo
secciones sobre los dilemas éticos. Fuentes: (Página 2, Chunk 1), (Página 1, Chunk 1)
----------------------------------------

--- Pregunta 2 ---

❓ Pregunta: ¿Cómo funciona el aprendizaje automático?
📚 Encontrados 3 chunks relevantes:
  1. Página 4, Chunk 1 (Score: 0.306)
  2. Página 5, Chunk 1 (Score: 0.357)
💡 Respuesta: Aprendizaje automático - Wikipedia, la enciclopedia libre
Fuente: h